In [ ]:
# Ame Digital - Teste Engenheiro de Dados
#
# Autor: Dennis Cardoso
#
# E-mail: dennis.cardoso@outlook.com
#
# Data: 22 de Dezembro de 2019

In [53]:
#Bibliotecas importação
import re
from datetime import datetime
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as functions

In [ ]:
# Iniciar spark context
conf = SparkConf().setMaster('local[*]')
sc = SparkContext().getOrCreate(conf)
sqlc =  SQLContext(sc)

In [176]:
# Declaração do Schema de dados a ser utilizado
schema = StructType([StructField('host',StringType(), True),StructField('timestamp',StringType(), True),StructField('request',StringType(), True),StructField('http_code',IntegerType(), True), StructField('total_bytes',IntegerType(), True)])

In [221]:
#Função que retorna data a partir de um string
def str_date(input_date):
        try:
            return datetime.fromordinal(input_date).strftime('%d/%m/%Y')
        except Exception as e:
            return '01/01/1900'

In [222]:
# Função para fazer parse dos dados de entrada
def parseLog(data):
        ''' Read and parse log data '''
        RE_MASK = '(.*) - - \[(.*):(.*):(.*):(.*)\] "(.*)" ([0-9]*) ([0-9]*|-)'

        try:
            re_result = re.compile(RE_MASK).match(data)
            host = re_result.group(1)
            ord_day = str_date(datetime.strptime(re_result.group(2), '%d/%b/%Y').toordinal())
            req = re_result.group(6)
            reply_code = int(re_result.group(7))
            
            try:
                reply_bytes = int(re_result.group(8))
            except ValueError as e:
                reply_bytes = 0
            return host, ord_day, req, reply_code, reply_bytes
        
        except Exception as e:
            return '', -1, '', -1, -1

In [223]:
# Teste da função Parse_log
valor = '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'
resultado = parseLog(valor)
resultado

('199.72.81.55', '01/07/1995', 'GET /history/apollo/ HTTP/1.0', 200, 6245)

In [202]:
# Gerar RDD com dados dos arquivos
rows = sc.textFile('files')

# Gerar parse dos dados de Log
nasa_parse = rows.map(parseLog)

# remover linhas com valores inválidos
nasa_rdd = nasa_parse.filter(lambda x: x[1] > -1)

In [203]:
# Criação do Dataframe
nasa_df = sqlc.createDataFrame(nasa_rdd, schema)

In [205]:
# Schema do dataframe
nasa_df.printSchema()
nasa_df.select('*').show()

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- request: string (nullable = true)
 |-- http_code: integer (nullable = true)
 |-- total_bytes: integer (nullable = true)

+--------------------+---------+--------------------+---------+-----------+
|                host|timestamp|             request|http_code|total_bytes|
+--------------------+---------+--------------------+---------+-----------+
|        199.72.81.55|   728475|GET /history/apol...|      200|       6245|
|unicomp6.unicomp.net|   728475|GET /shuttle/coun...|      200|       3985|
|      199.120.110.21|   728475|GET /shuttle/miss...|      200|       4085|
|  burger.letters.com|   728475|GET /shuttle/coun...|      304|          0|
|      199.120.110.21|   728475|GET /shuttle/miss...|      200|       4179|
|  burger.letters.com|   728475|GET /images/NASA-...|      304|          0|
|  burger.letters.com|   728475|GET /shuttle/coun...|      200|          0|
|     205.212.115.106|   728475|

In [206]:
# Número de HOSTs únicos
hosts_list = nasa_df.groupby('host').agg(functions.count('host').alias('count')).filter("count = 1").count()

In [207]:
# Total De error 404 dentro do Periodo
total_404_errors = nasa_df.filter("http_code = 404")
total_404_errors.count()

20901

In [163]:
# Quais dias do período especificado tiveram o maior número de erros 404.
periodo_404 = total_404_errors.withColumn("timestamp2",(functions.to_date(col("timestamp")),"DD-MM-YYYY")).show(false)

NameError: name 'col' is not defined

In [208]:
# O total de bytes retornados no período, com uma visão acumulada
total_404_errors.select('*').show()

+--------------------+---------+--------------------+---------+-----------+
|                host|timestamp|             request|http_code|total_bytes|
+--------------------+---------+--------------------+---------+-----------+
|dd15-062.compuser...|   728475|GET /news/sci.spa...|      404|          0|
|   netport-27.iu.net|   728475|GET /pub/winvn/re...|      404|          0|
|   netport-27.iu.net|   728475|GET /pub/winvn/re...|      404|          0|
|blv-pm0-ip28.halc...|   728475|GET /persons/astr...|      404|          0|
|blv-pm0-ip28.halc...|   728475|GET /persons/astr...|      404|          0|
|cu-dialup-1005.ci...|   728475|GET /pub/winvn/re...|      404|          0|
|cu-dialup-1005.ci...|   728475|GET /pub/winvn/re...|      404|          0|
|cu-dialup-1005.ci...|   728475|GET /pub/winvn/re...|      404|          0|
|    mimas.execpc.com|   728475|GET /shuttle/miss...|      404|          0|
|zoom112.telepath.com|   728475|GET /history/apol...|      404|          0|
|ad08-027.co